In [2]:
import json
from functools import lru_cache
from operator import itemgetter
from typing import List
import boto3

import pandas as pd
import requests
from pydantic import BaseModel, confloat, AnyUrl,conint
from ratelimit import RateLimitException, limits
from tenacity import *
from yarl import URL

In [3]:
API_ENDPOINT = URL("https://api.coingecko.com/api/v3")

In [4]:
session = boto3.Session(profile_name='master')

In [8]:
@retry(
    retry=retry_if_exception_type(requests.exceptions.RequestException),
    wait=wait_exponential(max=60),
    stop=stop_after_attempt(3),
)
@retry(
    retry=retry_if_exception_type(RateLimitException),
    wait=wait_fixed(60),
    stop=stop_after_delay(360),
)
@limits(calls=50, period=60)
def call_get(url: URL, params: dict) -> str:
    return requests.get(url.update_query(params)).text

In [9]:
class MarketDataInfo(BaseModel):
    id: str
    symbol: str
    current_price: float
    market_cap: int

In [6]:
# list_of_coins = pd.read_csv("/home/vlad/master_project/data/coin_list/list_of_coins.csv")['id'].to_list()

In [18]:
path = "s3://kosmobiker-masterproject/data/raw/"

In [36]:
import awswrangler as wr


In [5]:
s3 = session.client('s3')

In [38]:
s3_clientobj = s3.get_object(Bucket='kosmobiker-masterproject', Key='data/raw/ape_in_records_historical_prices.json')

In [39]:
d = s3_clientobj['Body'].read().decode('utf-8')

In [40]:
json_data = json.loads(d)

In [41]:
def get_json_from_s3(bucket, key, session):
    s3 = session.client('s3')
    obj = s3.get_object(Bucket=bucket, Key=key)
    data = obj['Body'].read().decode('utf-8')
    return json.loads(data)

In [57]:
# res = get_json_from_s3("kosmobiker-masterproject", 'data/raw/ape_in_records_historical_prices.json', session)
res = get_json_from_s3("kosmobiker-masterproject", 'data/raw/amateras_historical_prices.json', session)

In [68]:
def make_df(json_data):
    return pd.DataFrame({
                "timestamp" : [i[0] for i in json_data['prices']],
                "prices" : [i[1] for i in json_data['prices']],
                "market_caps" : [i[0] for i in json_data['market_caps']],
                "total_volumes" : [i[0] for i in json_data['total_volumes']]
    })
    


In [69]:
df = make_df(res)

In [74]:
res['prices']

[[1639991546619, 0.08086620615009682],
 [1639995033568, 0.0784355170814607],
 [1639998269817, 0.0787459036796956],
 [1640001846943, 0.07880882690482824],
 [1640005463432, 0.07891344702889956],
 [1640008841108, 0.07804258131130491],
 [1640012433380, 0.07869080707247689],
 [1640016034514, 0.07871326238705387],
 [1640019716005, 0.07907984681084211],
 [1640023303913, 0.07921267198131697],
 [1640026898703, 0.07897479488495393],
 [1640030502718, 0.07939318475643513],
 [1640034269653, 0.0801939474192529],
 [1640037868445, 0.08013852369546431],
 [1640041460576, 0.0800044973214381],
 [1640045063961, 0.08064460922251548],
 [1640049582061, 0.07967844116961825],
 [1640053127065, 0.07968579062164743],
 [1640056791745, 0.07981735285144893],
 [1640060385414, 0.0810250853899879],
 [1640063984676, 0.08108390283360142],
 [1640067556472, 0.08071053826121219],
 [1640071167902, 0.08076842790014299],
 [1640074759799, 0.08107056693882697],
 [1640078424735, 0.08132873561209732],
 [1640082006924, 0.08112581937